In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import PIL.Image as Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torchvision
from tqdm import tqdm
import time

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([transforms.Resize((256, 256)),
                                transforms.CenterCrop((224, 224)),
                              transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])

In [7]:
class val_imagenet_dataset(Dataset):   
    def __init__(self, root="../imagenet/", transform=None):
        self.transform = transform
        self.root = root
        self.data = []
        self.target = []
        f = open(self.root+"/ILSVRC2012_validation_ground_truth.txt")
        label_data = f.readlines()
        file_list = os.listdir(self.root)
        image_list = [image for image in file_list if image.endswith(".JPEG")]
        image_list.sort()
        for idx, label in enumerate(tqdm(label_data)):
            self.data.append(image_list[idx])
            self.target.append(int(label)-1) 
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        path, target = self.data[idx], self.target[idx]
        
        with open(self.root+path, 'rb') as f:
            img = Image.open(f)
            sample = img.convert('RGB')

        
        if self.transform is not None:
            sample = self.transform(sample)
        
        return sample, target
    

In [8]:
def val_model(model, dataloader, device, criterion =None):
    since = time.time()
    acc = 0
    loss = 0
    data_size = dataloader.dataset.__len__()
    for inputs, labels in tqdm(dataloader):
        model.eval()
        model.to(device)
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        acc += torch.sum(preds==labels.data)
        
        if criterion is not None:
            _loss = criterion(outputs, labels)
            loss +=_loss.item() * inputs.size(0)
            print("local loss : {:4f}".format(loss))
        
    acc = acc.double()/data_size
    loss = loss / data_size
    
    print("Acc : {:4f}".format(acc))
    if criterion is not None:
        print("Loss : {:4f}".format(loss))

In [9]:
val_imagenet = val_imagenet_dataset(transform=transform)
val_dataloader = DataLoader(val_imagenet, batch_size=1, shuffle=False, num_workers=2)

100%|██████████| 50000/50000 [00:00<00:00, 441639.57it/s]


In [ ]:
resnet_50_origin = torchvision.models.resnet50(pretrained=True)

val_model(resnet_50_origin, val_dataloader, device=device)

  4%|▎         | 1853/50000 [00:40<15:30, 51.75it/s] 